In [1]:
!pip install redshift_connector
!pip install unidecode
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.0/130.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=4f89bc6e2cb8ddee36113bd3fdd8f8cb7b2ba37683aab76448bb380c58a495a4
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [2]:
import pickle
import json
import redshift_connector
import pandas as pd
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)
import numpy as np
import re
import os

from unidecode import unidecode
from collections import Counter
from math import ceil
from langdetect import detect
from random import sample

## Support Files

Throughout the modeling process, some of the model artifacts needed to be updated and so this notebook was used to quickly update those files.

In [65]:
# location where current files are located
curr_model_artifacts_location = "./artifacts/"

# Load the needed files
with open(f"{curr_model_artifacts_location}departments_list.pkl", "rb") as f:
    departments_list = pickle.load(f)

print("Loaded list of departments")

with open(f"{curr_model_artifacts_location}countries_list_flat.pkl", "rb") as f:
    countries_list_flat = pickle.load(f)

print("Loaded flat list of countries")

with open(f"{curr_model_artifacts_location}countries.json", "r") as f:
    countries_dict = json.load(f)

print("Loaded countries dictionary")

with open(f"{curr_model_artifacts_location}city_country_list.pkl", "rb") as f:
    city_country_list = pickle.load(f)

print("Loaded strings of city/country combinations")

Loaded list of departments
Loaded flat list of countries
Loaded countries dictionary
Loaded strings of city/country combinations


### Looking at ROR

In [6]:
def get_geoname_admin(address_dict):
    try:
        geoname_admin = address_dict['geonames_city']['geonames_admin1']['name']
    except:
        geoname_admin = "None"

    return geoname_admin

In [7]:
def get_final_region(ror_state, ror_region):
    if isinstance(ror_state, str):
        return ror_state
    elif isinstance(ror_region, str):
        return ror_region
    else:
        return None

In [8]:
def check_for_backwards_univ(curr_names):
    names = curr_names.copy()
    for one_name in curr_names:
        split_name = one_name.split(" ")
        if len(split_name) == 3:
            if (split_name[0] == 'University') & (split_name[1] == 'of'):
                names.append(f"{split_name[2]} University")
        elif len(split_name) == 2:
            if (split_name[1] == 'University'):
                names.append(f"University of {split_name[0]}")
        else:
            pass
    return names

In [9]:
def add_names_to_list(all_names):
    names = all_names.copy()
    if "Harvard University" in names:
        names.append("Harvard Medical School")
    elif "University of Oxford" in names:
        names.append("Oxford University")
    else:
        pass

    return names

In [10]:
def get_exact_names(name, aliases, acronyms, labels):
    all_names = [name] + aliases + acronyms + [i['label'] for i in labels]
    all_names = add_names_to_list(all_names)
    all_names = [x for x in all_names if ~x.startswith('Department of')]
    all_names_clean = [string_match_clean(x) for x in all_names]
    return [x for x in all_names_clean if len(x) > 4]

In [11]:
def string_match_clean(text):
    #replace "&" with "and"
    if "r&d" not in text.lower():
        text = text.replace(" & ", " and ")

    # take country out
    if text.strip().endswith(")"):
        for country in countries_list_flat:
            if text.strip().endswith(f"({country})"):
                text = text.replace(f"({country})", "")

    # use unidecode
    text = unidecode(text.strip())

    # replacing common abbreviations
    text = text.replace("Univ.", "University")
    text = text.replace("Lab.", "Laboratory")

    # take out spaces, commas, dashes, periods, etcs
    text = re.sub("[^0-9a-zA-Z]", "", text)

    return text

In [12]:
def list_of_all_names(oa_name, ror_names, extra_names, use_extra_names=False):
    banned_names = ['UniversityHospital','Coastal','Brunswick','Continental']
    if isinstance(ror_names, list):
        pass
    else:
        ror_names = []

    if isinstance(oa_name, str):
        oa_string = [string_match_clean(oa_name)]
    else:
        oa_string = []

    if use_extra_names:
        if isinstance(extra_names, list):
            pass
        else:
            extra_names = []
    else:
        extra_names = []

    return [x for x in list(set(oa_string+ror_names+extra_names)) if
            ((len(x) > 4) & (x not in banned_names))]

In [77]:
# this file is not provided but the needed data is all institutions in OpenAlex
# with the following columns: 'ror_id','affiliation_id','display_name'
institutions_df = pd.read_parquet(f"{curr_model_artifacts_location}/OA_static_institutions_single_file.parquet")
#institutions_df = pd.read_json(f"{curr_model_artifacts_location}/OA_data.json")

In [78]:
institutions_df['ror_id'] = institutions_df['ror_id'].apply(lambda x: x.split("/")[-1])
institutions_df['affiliation_id'] = institutions_df['affiliation_id'].apply(lambda  x: x.split("I")[-1])
institutions_df= institutions_df[['affiliation_id', 'ror_id','display_name']]
institutions_df.head()

,affiliation_id,ror_id,display_name
0,151201029,03cqe8w59,Consejo Nacional de Investigaciones Científicas y Técnicas
1,24354313,0081fs513,University of Buenos Aires
2,874386039,01tjs6929,Universidad Nacional de La Plata
3,166401450,056tb7j80,Universidad Nacional de Córdoba
4,53241121,05rxmkq09,Fundación Ciencias Exactas y Naturales


In [ ]:
# institutions = institutions_df.set_index('affiliation_id').to_dict(orient='index')

In [67]:
#ror = pd.read_json("./v1.19-2023-02-16-ror-data.json")
temp_ror = pd.read_json(f"{curr_model_artifacts_location}/v1.50-2024-07-29-ror-data.json")
ror = temp_ror[temp_ror["country"].apply(lambda x: x.get("country_code") == "AR")]

In [69]:
ror['address'] = ror['addresses'].apply(lambda x: x[0])
ror['ror_id'] = ror['id'].apply(lambda x: x.split("/")[-1])
ror['types'] = ror['types'].apply(lambda x: x[0])

<ipython-input-69-7cec6f5f7829>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ror['address'] = ror['addresses'].apply(lambda x: x[0])
<ipython-input-69-7cec6f5f7829>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ror['ror_id'] = ror['id'].apply(lambda x: x.split("/")[-1])
<ipython-input-69-7cec6f5f7829>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [70]:
ror['country_code'] = ror['country'].apply(lambda x: x['country_code'])
ror['country_name'] = ror['country'].apply(lambda x: x['country_name'])
ror['city'] = ror['address'].apply(lambda x: x['city'])
ror['state'] = ror['address'].apply(lambda x: x['state'])
ror['region'] = ror['address'].apply(get_geoname_admin)

<ipython-input-70-8b6d7ad44cea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ror['country_code'] = ror['country'].apply(lambda x: x['country_code'])
<ipython-input-70-8b6d7ad44cea>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ror['country_name'] = ror['country'].apply(lambda x: x['country_name'])
<ipython-input-70-8b6d7ad44cea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [81]:
ror_to_join = ror[['ror_id','name','status','types','aliases','acronyms','labels','city',
                   'state','region','country_name']].copy()

ror_to_join.columns = ['ror_id','name','status','types','aliases','acronyms','labels','city',
                       'temp_state','temp_region','country']

In [82]:
ror_to_join['state'] = ror_to_join.apply(lambda x: get_final_region(x.temp_state, x.temp_region), axis=1)

In [98]:
inst_ror = ror_to_join.merge(institutions_df[['ror_id','affiliation_id','display_name']],
                             how='inner', on='ror_id')
inst_ror.shape

(331, 14)

#### Getting file of multi-institution names

In [99]:
multi_inst_names_df = ror_to_join[['ror_id','name']].merge(institutions_df[['ror_id','affiliation_id']],
                                                        how='left', on='ror_id') \
['name'].value_counts().reset_index()

In [ ]:
multi_inst_names_df.head(10)

In [115]:
print(multi_inst_names_df.head())
print(multi_inst_names_df.dtypes)



                                        name  count
0        National University of the Littoral      1
1       National Center for Genome Resources      1
2  Instituto de Investigaciones Psicológicas      1
3      Fundación Salud, Ciencia y Tecnologia      1
4             National University of Rosario      1
name     object
count     int64
dtype: object


In [106]:
multi_inst_names = multi_inst_names_df[multi_inst_names_df['name']>1].iloc[:,0].tolist()

In [107]:
print(multi_inst_names)

[]


In [39]:
multi_inst_names_ids = inst_ror[inst_ror['name'].isin(multi_inst_names)]['affiliation_id'].tolist()

NameError: name 'multi_inst_names' is not defined

In [ ]:
with open(f"{curr_model_artifacts_location}multi_inst_names_ids.pkl", "wb") as f:
    pickle.dump(multi_inst_names_ids, f)

### Getting Mapping of Inactive Institutions

There are institutions in ROR that are listed as "Withdrawn" or "Inactive". There was some thought to use the old data associated with these ROR IDs and apply them to successors but for this model, we decided to hold off on doing this because we were unsure if there would be a benefit to doing so. Therefore, the code is provided but this data was not used in building the model.

In [120]:
def get_successors_from_relationships(relationships):
    successors = []
    parents = []
    for relationship in relationships:
        if relationship['type'] == 'Successor':
            successors.append(relationship['id'].split("/")[-1])
        elif relationship['type'] == 'Parent':
            parents.append(relationship['id'].split("/")[-1])
        else:
            pass
    return [successors, parents]

In [121]:
def get_extra_names(ror_id):
    if ror_id in successor_dict.keys():
        extra_names = []
        for old_id in successor_dict[ror_id]['ror_id']:
            extra_names += old_name_data[old_id]['successor_names']

        extra_names = list(set(extra_names))
    else:
        extra_names = []

    return extra_names

In [122]:
withdrawn_or_inactive_df = ror[ror['status'].isin(['withdrawn','inactive'])].copy()
withdrawn_or_inactive_df.shape

(3, 23)

In [123]:
withdrawn_or_inactive_df['successors_parents'] = withdrawn_or_inactive_df['relationships'] \
    .apply(get_successors_from_relationships)

In [126]:
withdrawn_or_inactive_df['successors'] = withdrawn_or_inactive_df['successors_parents'].apply(lambda x: x[0])
withdrawn_or_inactive_df['parents'] = withdrawn_or_inactive_df['successors_parents'].apply(lambda x: x[1])

In [127]:
withdrawn_or_inactive_df['successor_len'] = withdrawn_or_inactive_df['successors'].apply(len)

In [128]:
to_add_to_successors = withdrawn_or_inactive_df[withdrawn_or_inactive_df['successor_len']==1].copy()
to_add_to_successors['successor'] = to_add_to_successors['successors'].apply(lambda x: x[0])
to_add_to_successors.shape

(3, 28)

In [129]:
to_add_to_successors['successor_names'] = to_add_to_successors.apply(lambda x: get_exact_names(x['name'],
                                                                                               x.aliases,
                                                                                               x.acronyms,
                                                                                               x.labels),
                                                                     axis=1)

In [130]:
old_name_data = to_add_to_successors.set_index('ror_id')[['successor_names']].to_dict(orient='index')

In [131]:
successor_dict = to_add_to_successors.groupby('successor')['ror_id'].apply(list).reset_index()\
    .set_index('successor').to_dict(orient='index')

### Getting ROR String Matching File and Affiliation Dictionary

In [132]:
inst_ror['extra_names'] = inst_ror['ror_id'].apply(get_extra_names)

In [133]:
inst_ror['exact_names'] = inst_ror.apply(lambda x: get_exact_names(x['name'], x.aliases,
                                                                         x.acronyms, x.labels), axis=1)

In [134]:
inst_ror['final_names'] = inst_ror.apply(lambda x: list_of_all_names(x.display_name, x.exact_names,
                                                                     x.extra_names,
                                                                     use_extra_names=False), axis=1)

In [135]:
new_affiliation_dict = inst_ror.set_index('affiliation_id')[['display_name','city','state',
                                                             'country','final_names','ror_id','types']] \
.to_dict(orient='index')

In [136]:
with open(f"{curr_model_artifacts_location}/full_affiliation_dict.pkl", "wb") as f:
    pickle.dump(new_affiliation_dict, f)

### Updating the city/country file

This file is used to check the affiliation string to make sure it doesn't exactly match up with a city/region/country combo with no additional information.

In [137]:
city_region_country = inst_ror.drop_duplicates(subset=['city','country']).copy()
city_region_country.shape

(59, 17)

In [138]:
new_city_country_list = list(set([f"{i}{j}" for i,j in zip(city_region_country['city'].tolist(),
                                   city_region_country['country'].tolist())] +
         [f"{i}{j}{k}"for i,j,k in zip(city_region_country['city'].tolist(),
                                             city_region_country['state'].tolist(),
                                             city_region_country['country'].tolist()) if j ] +
         [f"{i}{j}" for i,j in zip(city_region_country['state'].tolist(),
                                   city_region_country['country'].tolist()) if i] +
         [f"{i}" for i in city_region_country['country'].tolist()] +
         [f"{i}" for i in city_region_country['state'].tolist() if i]))

new_city_country_list = list(set([string_match_clean(x) for x in new_city_country_list]))

In [139]:
len(new_city_country_list)

149

In [ ]:
with open(f"{curr_model_artifacts_location}/city_country_list.pkl", "wb") as f:
    pickle.dump(new_city_country_list, f)

### Flat country file is up to date

Flat country file is needed to search for country in the string for the model.

In [ ]:
len(list(set(countries_list_flat)))

345

In [ ]:
all_countries = []
for i in countries_dict.values():
    all_countries += i

In [ ]:
len(list(set(all_countries)))

396

In [ ]:
with open(f"{curr_model_artifacts_location}/countries_list_flat.pkl", "wb") as f:
    pickle.dump(list(set(all_countries)), f)

### Departments list update

Takes the old department list and updates it with additional department names.

In [141]:
with open(f"{curr_model_artifacts_location}/Education_dept", 'r') as f:
    education_dept_begs = f.readlines()

education_dept_begs = list(set([x.rstrip('\n') for x in education_dept_begs]))

In [142]:
departments_list = ['Psychology','Nephrology','Other departments','Other Departments','Nursing & Midwifery',
                    'Literature and Creative Writing','Neuroscience','Engineering','Computer Science',
                    'Chemistry','Biology','Medicine']

In [143]:
new_departments_list = list(set(departments_list + education_dept_begs))

In [144]:
with open(f"{curr_model_artifacts_location}departments_list.pkl", "wb") as f:
    pickle.dump(new_departments_list, f)

### Make affiliation IDs integers

In [145]:
with open(f"{curr_model_artifacts_location}/affiliation_vocab.pkl", "rb") as f:
    affiliation_vocab_basic = pickle.load(f)

new_affiliation_vocab_basic = {int(i):int(j) for j,i in affiliation_vocab_basic.items()}

print("Loaded basic affiliation vocab")

Loaded basic affiliation vocab


In [146]:
with open(f"{curr_model_artifacts_location}/affiliation_vocab.pkl", "wb") as f:
    pickle.dump(new_affiliation_vocab_basic, f)